# Take Home Exam 2

In [74]:
# read user's data
import json
import pandas as pd

users=pd.read_csv(r"C:\Users\musht\takehome\takehome_users.csv",encoding = "ISO-8859-1")
users.head()


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [75]:
#read user engagement data
user_engage = pd.read_csv(r"C:\Users\musht\takehome\takehome_user_engagement.csv",encoding = "ISO-8859-1")


In [76]:
user_engage.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [77]:
user_engage.time_stamp = pd.to_datetime(user_engage.time_stamp)
user_engage.index=user_engage.time_stamp
user_engage.drop(labels='time_stamp',axis=1,inplace=True)
user_engage.head()

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1


In [78]:
#we can use grouper funcction here
#Group by user_id and resample to 1 week period, then add over seven day period
df_aggregate = user_engage.groupby([pd.Grouper(freq='W'),'user_id']).sum()

In [79]:
df_aggregate.head()

visited
time_stamp user_id         
2012-06-03 563            1
           1693           1
           1995           1
           2120           1
           2136           1

In [80]:
#Pull out  all user id's with a sum of 3 or more indicating an adopted user
df_adopt = df_aggregate[df_aggregate.visited>=3].unstack(level=1).melt()
df_adopt.head()

,None,user_id,value
0,visited,1693,3.0
1,visited,1693,NaN
2,visited,1693,3.0
3,visited,1693,NaN
4,visited,1693,3.0


In [81]:

# Need  all unique id
unique_id=df_adopt.user_id.unique()
unique_id

array([ 1693,   728, 11764, ...,  9870, 10746,  8460], dtype=int64)

In [82]:
# determine range for index
index_range=range(len(unique_id))

In [83]:
adopted_final=pd.DataFrame(unique_id,index=index_range, columns=['user_id'])
adopted_final.head()

,user_id
0,1693
1,728
2,11764
3,5297
4,6171


In [84]:
df_final=users.merge(adopted_final, how='inner', left_on='object_id', right_on='user_id')
df_final.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id
0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2
1,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,10
2,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN,20
3,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0,33
4,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN,42


In [85]:
# chalao amar code
deko=df_final.columns[0:4]
deko

Index(['object_id', 'creation_time', 'name', 'email'], dtype='object')

In [86]:
#drop unnecessary coloums
df_final=df_final.drop(deko, axis=1)


In [87]:
df_final=df_final.drop('user_id', axis=1)


In [88]:
df_final.head()

,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,SIGNUP,1.401364e+09,0,0,58,NaN
3,GUEST_INVITE,1.401518e+09,0,0,401,79.0
4,SIGNUP,1.401045e+09,1,0,235,NaN


In [89]:
# fill na values
#fill na values in invited_by_user column
df_final['invited_by_user_id'].fillna(value=0,inplace=True)

In [90]:
# create dummies
df_final=pd.get_dummies(df_final)
df_final.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1.396238e+09,0,0,1,316.0,0,1,0,0,0
1,1.401833e+09,1,1,318,4143.0,0,1,0,0,0
2,1.401364e+09,0,0,58,0.0,0,0,0,1,0
3,1.401518e+09,0,0,401,79.0,1,0,0,0,0
4,1.401045e+09,1,0,235,0.0,0,0,0,1,0


In [91]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#scale data
scaler = StandardScaler()
features = scaler.fit_transform(df_final)

#fit PCA
pca = PCA()
components = pca.fit_transform(features)

C:\Users\musht\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\musht\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [92]:
import numpy as np
np.sum(pca.explained_variance_ratio_[0:6])

0.8282488696342485

In [93]:
# Now find influencial feature
df_influencial = pd.DataFrame(pca.components_,columns=df_features.columns,index=['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10'])
#absolute values of correlation with principal components
best_features = np.absolute(df_influencial[np.absolute(df_influencial) > 0.1])


In [94]:
#sum correlations to get a relative estimate of the feature importance
best_features.head(6).sum(axis=0).sort_values(ascending=False)

SIGNUP_GOOGLE_AUTH            1.548742
PERSONAL_PROJECTS             1.438816
SIGNUP                        1.335089
last_session_creation_time    1.088250
ORG_INVITE                    1.076612
org_id                        1.055271
GUEST_INVITE                  1.016688
enabled_for_marketing_drip    0.698910
opted_in_to_mailing_list      0.695712
invited_by_user_id            0.637653
dtype: float64

The first six principal components account for 80%  of the data variation in the dataset of adopted users, which suggests that these variables are highly influencial of adopted users. 